In [43]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm
import pandas as pd
from gensim.models import KeyedVectors
from numpy import array


In [7]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
model = KeyedVectors.load_word2vec_format('model-whole.bin', binary = True)

In [19]:
def MSW(word,n):
    t = dict(model.most_similar(word, topn=n))
    return list(t.keys())

In [107]:
mtf = list()

for word in MSW('kabinet', 10):
    
    word_list = list()
    
    lw = MSW(word, 10)
    
    for w in lw:
        tmp = MSW(w, 10)
        for i in tmp:
            word_list.append(i)
    
    for x in word_list:
        mtf.append(x)
list(set(mtf))

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['bill',
 'traktaat',
 'nota',
 'dupuy',
 'parlement',
 'ziccing',
 'vredesconferentie',
 'generaal',
 'liberalismus',
 'jules',
 'centrum',
 'hertog',
 'regeringsstelsel',
 'caprivi',
 'international',
 'minifters',
 'ultimatum',
 'vredesonderhandelingen',
 'mijnwezen',
 'stuur',
 'staatssecretaris',
 'collegie',
 'eegeering',
 'gematigde',
 'ferdinand',
 'element',
 'onderwijzersgenootschap',
 'magt',
 'bisschoppen',
 'collectieve',
 'regentschap',
 'minisiers',
 'gouvernement',
 'salisbury',
 'ministor',
 'presidenc',
 'vatikaan',
 'verdrag',
 'kongres',
 'gemeentebestuur',
 'vorstendom',
 'russell',
 'onderhandelingen',
 'conferentien',
 'prestige',
 'grey',
 'premier',
 'bijbelgenootschap',
 'oudminister',
 'verbond',
 'molé',
 'rijksdag',
 'minister',
 'bestrijder',
 'internationalen',
 'miniseer',
 'regering',
 'grondbeginselen',
 'leerstuk',
 'committé',
 'conferentiën',
 'koncilie',
 'chamberlain',
 'hoofdcomité',
 'iecs',
 'tarief',
 'instituut',
 'john',
 'conferentie',
 'ti

In [100]:
mtf = np.average(model[list(set(mtf))], axis=0)

In [101]:
model.add('stab_av',mtf,replace=False)


In [104]:
model.distance('instab_av', 'stab_av')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.2114536166191101